In [1]:
import sys
import json
sys.path.append('./../')
sys.path.append('./../tirg/')
from main import *

In [2]:
opt = parse_opt() 
opt.batch_size = 32
opt.coco_path = '../../../../datasets/coco'
opt.sic112_path = '../../../../datasets/SIC112/'

logger = SummaryWriter(comment = opt.comment)

trainset, _, sic112 = load_datasets(opt)

17919 745
sucessfully loaded features
sucessfully loaded features
sucessfully loaded features


In [3]:
# add subject, verb and location annotations to SIC112
for img in sic112.imgs:
    img['subjects'] = [img['captions'][0].split()[0]]
    if img['captions'][0].split()[1].endswith("ing"):
        img['verbs'] = [img['captions'][0].split()[1]]
        img['locations'] = [' '.join(img['captions'][0].split()[2:])]
    else:
        img['verbs'] = []
        img['locations'] = [' '.join(img['captions'][0].split()[1:])]

In [4]:
# add subject, verb and location annotations to coco train 2014
# (need 'coco_splitted_captions_train2014.json' preprocess_coco first)
id2img = {}
for img in trainset.imgs:
    id2img[img['id']] = img
    img['subjects'] = []
    img['verbs'] = []
    img['locations'] = []
for caption in tqdm(json.load(open('coco_splitted_captions_train2014.json', 'rt'))['annotations']):
    img = id2img[caption['image_id']]
    if caption['subject_phrase'] is not None:
        img['subjects'] += [caption['subject_phrase']]
    if caption['verb_phrase'] is not None:
        img['verbs'] += [caption['verb_phrase']]
    if caption['location_phrase'] is not None:
        img['locations'] += [caption['location_phrase']]
        
# update trainset.__getitem__
#trainset.old_get = trainset.__getitem__
#def new_get(self, idx):
#    item = self.old_get(idx)
#type(trainset).__getitem__ = type(type(trainset).__getitem__)(new_get, trainset, type(trainset))

100%|██████████| 414113/414113 [00:01<00:00, 290800.53it/s]


In [5]:
class One2OneTransformation(torch.nn.Module):
    def __init__(self):
        super(One2OneTransformation, self).__init__()
        embed_dim = opt.embed_dim
        self.m = torch.nn.Sequential(
            torch.nn.Linear(embed_dim * 1, embed_dim * 2),
            torch.nn.ReLU(),
            torch.nn.Linear(embed_dim * 2, embed_dim * 2),
            torch.nn.BatchNorm1d(embed_dim * 2),
            torch.nn.ReLU(),
            torch.nn.Linear(embed_dim * 2, embed_dim)
        )
        self.norm = torch_functions.NormalizationLayer(learn_scale=False)

    def forward(self, x):
        f = self.norm(x)
        f = self.m(f)
        return f
    
class Three2OneTransformation(torch.nn.Module):
    def __init__(self):
        super(Three2OneTransformation, self).__init__()
        embed_dim = opt.embed_dim
        self.m = torch.nn.Sequential(
            torch.nn.Linear(embed_dim * 3, embed_dim * 5),
            torch.nn.ReLU(),
            torch.nn.Linear(embed_dim * 5, embed_dim * 5),
            torch.nn.BatchNorm1d(embed_dim * 5),
            torch.nn.ReLU(),
            torch.nn.Linear(embed_dim * 5, embed_dim)
        )
        self.norm = torch_functions.NormalizationLayer(learn_scale=False)

    def forward(self, x):
        f = torch.cat([self.norm(i) for i in x], dim=1)
        f = self.m(f)
        return f

model = create_model(opt, trainset)
model.subject_extractor = One2OneTransformation()
model.verb_extractor = One2OneTransformation()
model.location_extractor = One2OneTransformation()
model.svl_combine = Three2OneTransformation() 
model = model.cuda()
optimizer = create_optimizer(opt, model)

In [6]:
def test_svl(model, testset, opt):
    model = model.eval()

    # all img features
    img_features = []
    for data in testset.get_loader(batch_size = opt.batch_size, shuffle = False, drop_last= False):
        # extract image features
        imgs = np.stack([d['image'] for d in data])
        imgs = torch.from_numpy(imgs).float()
        if len(imgs.shape) == 2:
            imgs = model.img_encoder.fc(imgs.cuda())
        else:
            imgs = model.img_encoder(imgs.cuda())
        imgs = model.snorm(imgs).cpu().detach().numpy()
        img_features += [imgs]

    img_features = np.concatenate(img_features, axis=0)
    img_labels = [img['captions'][0] for img in testset.imgs]
    
    # construct random queries
    queries = []
    np.random.seed(123)
    for _ in range(5):
      for img in testset.imgs:
        if len(img['verbs']) == 0:
            continue
        while True:
            i = np.random.randint(0, len(testset.imgs))
            if img['subjects'][0] == testset.imgs[i]['subjects'][0] and img is not testset.imgs[i]:
                break
        while True:
            j = np.random.randint(0, len(testset.imgs))
            if len(testset.imgs[j]['verbs']) == 0:
                continue
            if img['verbs'][0] == testset.imgs[j]['verbs'][0] and img is not testset.imgs[j]:
                break
        while True:
            k = np.random.randint(0, len(testset.imgs))
            if img['locations'][0] == testset.imgs[k]['locations'][0] and img is not testset.imgs[k]:
                break
            
        
        queries += [{
            'subject_img_id': i,
            'verb_img_id': j,
            'location_img_id': k,
            'subject': img['subjects'][0],
            'verb': testset.imgs[j]['verbs'][0],
            'location': img['locations'][0],
            'label': img['captions'][0]
        }]
        
    #----
    #----
    r = []
    query_setting_combinations = []
    for s in ['t', 'i']:
        for v in ['t', 'i']:
            for l in ['t', 'i']:
                query_setting_combinations += [(s, v, l)]
    for s, v, l in query_setting_combinations:
        # compute query features
        query_features = []
        query_labels = []
        for i in range(0, len(queries), opt.batch_size):
            if s == 'i':
                subjects = model.subject_extractor(torch.from_numpy(
                    img_features[[q['subject_img_id'] for q in queries[i:(i+opt.batch_size)]],:]
                ).cuda())
            else:
                subjects = model.text_encoder([q['subject'] for q in queries[i:(i+opt.batch_size)]])
            if v == 'i':
                verbs = model.verb_extractor(torch.from_numpy(
                    img_features[[q['verb_img_id'] for q in queries[i:(i+opt.batch_size)]],:]
                ).cuda())
            else:
                verbs = model.text_encoder([q['verb'] for q in queries[i:(i+opt.batch_size)]])
            if l == 'i':
                locations = model.location_extractor(torch.from_numpy(
                    img_features[[q['location_img_id'] for q in queries[i:(i+opt.batch_size)]],:]
                ).cuda())
            else:
                locations = model.text_encoder([q['location'] for q in queries[i:(i+opt.batch_size)]])
            svl = model.svl_combine([subjects, verbs, locations])
            svl = svl.cpu().detach().numpy()
            query_features += [svl]
            query_labels += [q['label'] for q in queries[i:(i+opt.batch_size)]]

        query_features = np.concatenate(query_features, axis=0)

        # compute recall
        def measure_retrieval_performance(query_features, name = 'X'):
            sims = query_features.dot(img_features.T)
            sims = sims
            for k in [1, 5, 10]:
                r1 = 0.0
                r1_novel = 0.0
                count_novel = 0.0
                r1_nonnovel = 0.0
                count_nonnovel = 0.0
                for i in range(sims.shape[0]):
                    novel_query = False
                    if queries[i]['label'].split()[0] in ['trex', 'stormtrooper', 'darthvader', 'chewbacca']:
                        novel_query = True
                    if novel_query:
                        count_novel += 1
                    else:
                        count_nonnovel += 1
                        
                    s = -sims[i,:]
                    s = np.argsort(s)
                    if query_labels[i] in [img_labels[s[j]] for j in range(k)]:
                        r1 += 1
                        if novel_query:
                            r1_novel += 1
                        else:
                            r1_nonnovel += 1
                        
                r1 /= sims.shape[0]
                r.append(('svl_' + name + '_recall_top' + str(k), r1))
            return r
        measure_retrieval_performance(query_features, name = s + v + l)
    return r

def test(model, testset, opt):
    n = 1100
    if len(testset) < 10000:
        n = len(testset)
    r = test_text_to_image_retrieval(model, testset, opt, n)
    if '112' in testset.name():
        r += test_svl(model, testset, opt)
    return r

In [7]:
def compute_losses(model, data, losses_tracking, add_extract_compose_losses = True):
    losses = []

    # joint embedding loss
    imgs = np.stack([d['image'] for d in data])
    imgs = torch.from_numpy(imgs).float()
    if len(imgs.shape) == 2:
        imgs = model.img_encoder.fc(imgs.cuda())
    else:
         imgs = model.img_encoder(imgs.cuda())
    texts = [random.choice(d['captions']) for d in data]
    texts = model.text_encoder(texts)
    loss_name = 'joint_embedding'
    loss_weight = 1
    loss_value = model.pair_loss(texts, imgs).cuda()
    losses += [(loss_name, loss_weight, loss_value)]
    
    def do_add_extract_compose_losses():
        try:
            subjects = [random.choice(trainset.imgs[d['index']]['subjects']) for d in data]
            verbs = [random.choice(trainset.imgs[d['index']]['verbs']) for d in data]
            locations = [random.choice(trainset.imgs[d['index']]['locations']) for d in data]
        except:
            return
        encoded_subjects = model.text_encoder(subjects).detach()
        encoded_verbs = model.text_encoder(verbs).detach()
        encoded_locations = model.text_encoder(locations).detach()
        extracted_subjects = model.subject_extractor(random.choice([texts, imgs]).detach())
        extracted_verbs = model.verb_extractor(random.choice([texts, imgs]).detach())
        extracted_location = model.location_extractor(random.choice([texts, imgs]).detach())
            
        # extract
        loss_value = 0
        loss_value += model.pair_loss(
            torch.cat([extracted_subjects, extracted_verbs, extracted_location]),
            torch.cat([encoded_subjects, encoded_verbs, encoded_locations])
        ).cuda()
        loss_name = 'extract'
        loss_weight = 1
        losses.append((loss_name, loss_weight, loss_value))
        
        # compose with encoded
        loss_value = model.pair_loss(
            model.svl_combine([encoded_subjects, encoded_verbs, encoded_locations]),
            random.choice([imgs, model.text_encoder([s + ' ' + v + ' ' + l for s, v, l in zip(subjects, verbs, locations)])]).detach()
        ).cuda()
        loss_name = 'compose1'
        loss_weight = 0.5
        losses.append((loss_name, loss_weight, loss_value))

        # shuffle
        shuffled_subjects_indices = range(len(data))
        shuffled_verbs_indices = range(len(data))
        shuffled_locations_indices = range(len(data))
        random.shuffle(shuffled_subjects_indices)
        random.shuffle(shuffled_verbs_indices)
        random.shuffle(shuffled_locations_indices)
        encoded_subjects = encoded_subjects[shuffled_subjects_indices,:]
        encoded_verbs = encoded_verbs[shuffled_verbs_indices,:]
        encoded_locations = encoded_locations[shuffled_locations_indices,:]
        extracted_subjects = extracted_subjects[shuffled_subjects_indices,:]
        extracted_verbs = extracted_verbs[shuffled_verbs_indices,:]
        extracted_location = extracted_location[shuffled_locations_indices,:]
        subjects = np.array(subjects)[shuffled_subjects_indices]
        verbs = np.array(verbs)[shuffled_verbs_indices]
        locations = np.array(locations)[shuffled_locations_indices]

        # compose with extracted
        loss_value = model.pair_loss(
            model.svl_combine([extracted_subjects, extracted_verbs, extracted_location]),
            model.text_encoder([s + ' ' + v + ' ' + l for s, v, l in zip(subjects, verbs, locations)]).detach()
        ).cuda()
        loss_name = 'compose2'
        loss_weight = 0.5
        losses.append((loss_name, loss_weight, loss_value))
    if add_extract_compose_losses:
        do_add_extract_compose_losses()

    # total loss
    total_loss = sum([loss_weight * loss_value for loss_name, loss_weight, loss_value in losses])
    assert(not torch.isnan(total_loss))
    losses += [('total training loss', None, total_loss)]

    # save losses
    for loss_name, loss_weight, loss_value in losses:
        if not losses_tracking.has_key(loss_name):
            losses_tracking[loss_name] = []
        losses_tracking[loss_name].append(float(loss_value.data.item()))
    return total_loss

def train_1_epoch(model, optimizer, trainset, opt, losses_tracking, add_extract_compose_losses = True):
    model.train()
    loader = trainset.get_loader(
        batch_size=opt.batch_size, shuffle=True,
        drop_last=True, num_workers=opt.loader_num_workers)
    for data in tqdm(loader, desc = 'training 1 epoch'):
        total_loss = compute_losses(model, data, losses_tracking, add_extract_compose_losses)
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

In [8]:
# train loop
losses_tracking = {}
epoch = 0
tic = time.time()
while True:

    # show stat, training losses
    print 'Epoch', epoch, 'Elapsed time', round(time.time() - tic, 4)
    tic = time.time()
    for loss_name in losses_tracking:
        avg_loss = np.mean(losses_tracking[loss_name][-250:])
        print '   ', loss_name, round(avg_loss, 4)
        logger.add_scalar(loss_name, avg_loss, epoch)

    # test
    tests = []
    for dataset in [trainset, sic112]:
        t = test(model, dataset, opt)
        tests += [(dataset.name() + ' ' + metric_name, metric_value) for metric_name, metric_value in t]
    for metric_name, metric_value in tests:
        print ' ', metric_name, round(metric_value, 4)
        logger.add_scalar(metric_name, metric_value, epoch)

    # train
    if epoch >= opt.num_epochs:
        break
    train_1_epoch(model, optimizer, trainset, opt, losses_tracking,
                  add_extract_compose_losses = epoch>=1)
    epoch += 1

    # learing rate scheduling
    if epoch % opt.learning_rate_decay_frequency == 0:
        for g in optimizer.param_groups:
            g['lr'] *= 0.1

Epoch 0 Elapsed time 0.0012


training 1 epoch:   0%|          | 0/2586 [00:00<?, ?it/s]

  CocoCapTrain text2image_recall_top1 0.1289
  SimpleImageCaptions112 text2image_recall_top1 0.1578
  SimpleImageCaptions112 svl_ttt_recall_top1 0.0
  SimpleImageCaptions112 svl_ttt_recall_top5 0.0368
  SimpleImageCaptions112 svl_ttt_recall_top10 0.0614
  SimpleImageCaptions112 svl_tti_recall_top1 0.0
  SimpleImageCaptions112 svl_tti_recall_top5 0.0368
  SimpleImageCaptions112 svl_tti_recall_top10 0.0737
  SimpleImageCaptions112 svl_tit_recall_top1 0.0
  SimpleImageCaptions112 svl_tit_recall_top5 0.0246
  SimpleImageCaptions112 svl_tit_recall_top10 0.0614
  SimpleImageCaptions112 svl_tii_recall_top1 0.0
  SimpleImageCaptions112 svl_tii_recall_top5 0.0268
  SimpleImageCaptions112 svl_tii_recall_top10 0.0737
  SimpleImageCaptions112 svl_itt_recall_top1 0.0
  SimpleImageCaptions112 svl_itt_recall_top5 0.0246
  SimpleImageCaptions112 svl_itt_recall_top10 0.0562
  SimpleImageCaptions112 svl_iti_recall_top1 0.0
  SimpleImageCaptions112 svl_iti_recall_top5 0.0246
  SimpleImageCaptions112 svl_

training 1 epoch: 100%|██████████| 2586/2586 [02:53<00:00, 14.88it/s]


Epoch 2 Elapsed time 209.0682
    total training loss 3.1741
    extract 2.7595
    joint_embedding 0.6088
    compose2 1.453
    compose1 0.4708


...

Epoch 18 Elapsed time 211.3846
    total training loss 2.4167
    extract 2.3539
    joint_embedding 0.3611
    compose2 0.9492
    compose1 0.2573


training 1 epoch:   0%|          | 0/2586 [00:00<?, ?it/s]

  CocoCapTrain text2image_recall_top1 0.2829
  SimpleImageCaptions112 text2image_recall_top1 0.3059
  SimpleImageCaptions112 svl_ttt_recall_top1 0.3419
  SimpleImageCaptions112 svl_ttt_recall_top5 0.7465
  SimpleImageCaptions112 svl_ttt_recall_top10 0.9036
  SimpleImageCaptions112 svl_tti_recall_top1 0.2366
  SimpleImageCaptions112 svl_tti_recall_top5 0.6182
  SimpleImageCaptions112 svl_tti_recall_top10 0.7627
  SimpleImageCaptions112 svl_tit_recall_top1 0.2502
  SimpleImageCaptions112 svl_tit_recall_top5 0.6276
  SimpleImageCaptions112 svl_tit_recall_top10 0.7804
  SimpleImageCaptions112 svl_tii_recall_top1 0.1926
  SimpleImageCaptions112 svl_tii_recall_top5 0.4928
  SimpleImageCaptions112 svl_tii_recall_top10 0.65
  SimpleImageCaptions112 svl_itt_recall_top1 0.2236
  SimpleImageCaptions112 svl_itt_recall_top5 0.6044
  SimpleImageCaptions112 svl_itt_recall_top10 0.7546
  SimpleImageCaptions112 svl_iti_recall_top1 0.198
  SimpleImageCaptions112 svl_iti_recall_top5 0.5213
  SimpleImageC

training 1 epoch:  45%|████▌     | 1168/2586 [01:19<01:39, 14.27it/s]

KeyboardInterrupt: 